In [2]:
import os
from dotenv import load_dotenv

import textwrap

from langchain_community.graphs import Neo4jGraph
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain

load_dotenv()

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)


In [3]:
company_rows = kg.query("""
  MATCH (com:Company)
  RETURN com { .cusip6, .companyName } as company
""")

companies = list(map(lambda row: row['company'], company_rows))

print(companies[0])

{'cusip6': '64110D', 'companyName': 'NETAPP INC'}


In [4]:
import csv

cik_mappings= []

with open('./data/cusip6-to-cik-mapping.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader: # each row will be a dictionary
      cik_mappings.append(row)

first_cik_mapping = cik_mappings[0]

first_cik_mapping

{'cusip6': '001055',
 'cik': '4977',
 'names': "{'AFLAC INC'}",
 'cusip': "{'001055902', '000105510', '001055102', '001055952'}"}

In [5]:
def find(lst, key, value):
    for i, dic in enumerate(lst):
        if dic[key] == value:
            return i
    return -1

mapping_idx = find(cik_mappings, 'cusip6', '64110D')

cik_mappings[mapping_idx]

{'cusip6': '64110D',
 'cik': '1002047',
 'names': "{'Netapp Inc', 'NETAPP INC'}",
 'cusip': "{'64110D954', '64110D904', '64110D104', '64110d104'}"}

In [6]:
for company in companies:
    mapping_idx = find(cik_mappings, 'cusip6', company['cusip6'])
    if mapping_idx != -1:
        company['cik'] = cik_mappings[mapping_idx]['cik']
    else:
        company['cik'] = None

print(companies[0])

{'cusip6': '64110D', 'companyName': 'NETAPP INC', 'cik': '1002047'}


In [7]:
set_company_cik_cypher = """
  MATCH (com:Company {cusip6: $cusip6})
  SET com.cik = $cik
"""

for company in companies:
    if company['cik']:
        kg.query(set_company_cik_cypher, params={
            "cusip6": company['cusip6'],
            "cik": company['cik']
        })

In [31]:
from urllib.request import urlopen 
import urllib.request
import urllib.error

import json 

base_url = "https://data.sec.gov/api/xbrl/companyfacts"
headers = {'User-Agent': 'Andreas Kollegger andreas.kollegger@neo4j.com'}

def show_keys(dict):
    for key in dict.keys():
        print(key)

def show_company_info(company):
    print(f"Entity Name: {company['entityName']}")
    print(f"CIK: {company['cik']}")

for company in companies[0:1]:
    if company['cik']:
        company_cik = company['cik']
        company_url = f"{base_url}/CIK{company_cik.rjust(10,'0')}.json"
        company_request = urllib.request.Request(company_url, None, headers)

        print(company_url)
        try:
            with urlopen(company_request) as response:
                data = response.read()
                data_json = json.loads(data) 
                # show_keys(data_json)
                show_company_info(data_json)
                show_keys(data_json['facts'])
                show_keys(data_json['facts']['dei'])
                show_keys(data_json['facts']['us-gaap'])
        except urllib.error.HTTPError as e:
            print(f"HTTP Error: {e.code} {e.reason}")
        except urllib.error.URLError as e:
            print(e.reason)
        except Exception as e:
            print(e)
          


https://data.sec.gov/api/xbrl/companyfacts/CIK0001002047.json
Entity Name: NetApp, Inc.
CIK: 1002047
dei
us-gaap
EntityCommonStockSharesOutstanding
EntityPublicFloat
AcceleratedShareRepurchasesFinalPricePaidPerShare
AcceleratedShareRepurchasesSettlementPaymentOrReceipt
AccountsPayableCurrent
AccountsReceivableNetCurrent
AccruedIncomeTaxesNoncurrent
AccruedLiabilitiesCurrent
AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment
AccumulatedOtherComprehensiveIncomeLossAvailableForSaleSecuritiesAdjustmentNetOfTax
AccumulatedOtherComprehensiveIncomeLossCumulativeChangesInNetGainLossFromCashFlowHedgesEffectNetOfTax
AccumulatedOtherComprehensiveIncomeLossDefinedBenefitPensionAndOtherPostretirementPlansNetOfTax
AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax
AccumulatedOtherComprehensiveIncomeLossNetOfTax
AcquiredFiniteLivedIntangibleAssetAmount
AdditionalPaidInCapitalCommonStock
AdjustmentForAmortization
AdjustmentsNoncashItemsToReconcile

In [ ]:
# Create a langchain vector store from the existing Neo4j knowledge graph.
neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)

# Create a retriever from the vector store
retriever = neo4j_vector_store.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), chain_type="stuff", retriever=retriever
)


In [ ]:
chain(f"Where is Fedex headquartered?")

In [ ]:
set_company_location_cypher = """
  MATCH (com:Company {cusip6: $cusip6})
  SET com.companyAddress = $companyAddress
  SET com.location = point({latitude: $latitude, longitude: $longitude})
  MERGE (addr:Address {city: $city, state: $state} )
   ON CREATE SET addr.country = $country
  MERGE (com)-[:LOCATED_AT]->(addr)
"""

for company in companies:
    result = chain(f"Where is {company['companyName']} headquartered?")
    address_statement = result['answer']
    address_geocodes = gmaps.geocode(address_statement)
    if len(address_geocodes) > 0:
        address_geocode = address_geocodes[0]

        cusip6 = company['cusip6']

        location = get_location(address_geocode)
        city = get_city(address_geocode)
        state = get_state(address_geocode)
        postal = get_postalcode(address_geocode)
        country = get_country(address_geocode)

        cityOrState = city if city else state
        stateOrCountry = state if state else country 

        print_address(company['companyName'], address_geocode['formatted_address'],
            long_name(cityOrState), 
            long_name(stateOrCountry), 
            long_name(postal), 
            long_name(country)
        )

        if location and cityOrState and stateOrCountry:
            kg.query(set_company_location_cypher, params={
                "companyAddress": address_geocode['formatted_address'],
                "cusip6": company['cusip6'],
                "latitude": location['lat'],
                "longitude": location['lng'],
                "city": long_name(cityOrState),
                "state": long_name(stateOrCountry),
                "country": long_name(country)
            })
    else:
        print(f"no geocode found for {company['companyName']} at {address_statement}")

In [ ]:
# Which state has the most public companies listed?
kg.query("""
  MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
  RETURN address.state as state, count(address.state) as numCompanies
    ORDER BY numCompanies DESC
""")

In [ ]:
# Which city in California has the most companies listed?
kg.query("""
  MATCH p=(:Company)-[:LOCATED_AT]->(address:Address)
         WHERE address.state = 'California'
  RETURN address.city as city, count(address.city) as numCompanies
    ORDER BY numCompanies DESC
""")

In [ ]:
kg.query("""
  CALL db.index.fulltext.queryNodes("fullTextCompanyNames", $companyName) 
    YIELD node, score
  WITH node as com
  RETURN com.location, com.companyAddress
""", params={"companyName": "Fedex"})

In [ ]:
# Which investment firms are near Fedex?
kg.query("""
  CALL db.index.fulltext.queryNodes("fullTextCompanyNames", $companyName) 
         YIELD node, score
  WITH node as com
  MATCH (mgr:Manager)
    WHERE point.distance(mgr.location, com.location) < $withinMeters
  WITH mgr, 
    toInteger(point.distance(mgr.location, com.location) / 1000) as distanceKm
  RETURN mgr.managerName, mgr.managerAddress, 
        apoc.number.format(distanceKm) + "km" as distance

""", params={"companyName": "Fedex", "withinMeters": 100 * 1000})